# Planification de la tournée d’une infirmière.


Une infirmière doit s’occuper de plusieurs patients à domicile. Pour lui éviter des frais importants et un temps de travail allongé, on cherche à réduire son temps de trajet. Il s’agit donc de décider à l’avance l’ordre dans lesquels elle visitera ses patients. Nous faisons ici quelques hypothèses de travail, hypothèses qui pourront être modifiées en fonction de l’avancement des travaux :

- aucun patient n’est prioritaire ;
- le temps de trajet est proportionnel à la distance parcourue, la constante de proportionnalité ne dépend pas du moment de la journée (il n’y a pas de bouchons !) ;
- les patients restent chez eux toute la journée, et sont disponibles pour les soins toute la journée ;
- l’infirmière débute une tournée en partant de chez elle et la termine en rentrant chez elle. Les trajets aller et retour doivent bien sûr être comptés.

Les adresses des patients seront fournies sous la forme de deux coordonnées : la latitude et la longitude (des angles en degrés).

## Algorithmes pour planifier la tournée

Nous proposons plusieurs algorithmes de planification :
- trajet aléatoire amélioré petit à petit
- approche gloutonne.

Ces algorithmes sont détaillés dans la suite. Les coordonnées géographiques des patients sont données dans un objet `données` de type `Données`. 

- On peut récupérer les coordonnées géographiques de `patient` avec l'instruction `données.coordonnées(patient)`. On obtient alors un couple : le premier élément est la latitude, et le deuxième la longitude, toutes deux exprimées en degré.
- On peut trouver le nombre de patients avec l'instruction `données.nombre_patients()`.

- On construit un trajet en partant d'un trajet initial dont le départ et l'arrivée correspondent au domicile de l'infirmière, que l'on
  modifie peu à peu en ajoutant un patient à la fois.
  - Pour initialiser un trajet, on écrit `trajet = Trajet(données)` où `données` correspond à un jeu de données ;
  - pour ajouter un patient à `trajet`, on écrit `trajet.insère_patient(patient)`, ce qui a pour effet de modifier le trajet, mais aussi de marquer le patient comme vu ;
  - on peut afficher les patients non visités avec l'instruction `trajet.patients_à_visiter()`.

Manipulez ces fonctions en exécutant le code ci-dessous. Essayez de bien comprendre ce qu'il se passe, car ensuite ce sera à vous d'utiliser ces fonctions.

In [1]:
from Trajet import *

# on importe des données bidon pour tester les fonctions. Ces dernières sont déjà créées…
données = données_test

# on initialise un trajet : le point de départ et d'arrivée correspondent au domicile de l'infirmière.
trajet = Trajet(données)

# on vérifie :
print(trajet)

infirmière -> infirmière


In [2]:
# Où habite l'infirmière ?
données.coordonnées("infirmière")

(47.21811, -1.55448)

In [3]:
# Combien de patients à voir ?
données.nombre_patients()

3

In [4]:
# Quels sont les patients à voir ?
trajet.patients_à_visiter()

Patients à voir : 
- Marcel
- Françoise
- Timéo


In [5]:
# On décide arbitrairement que l'infirmière verra Françoise d'abord.
trajet.insère_patient("Françoise")

In [6]:
# Le trajet est modifié : Françoise doit correspondre au premier arrêt.
print(trajet)

infirmière -> Françoise -> infirmière


In [7]:
# Françoise ne doit plus apparaître parmi les patients à visiter.
trajet.patients_à_visiter()

Patients à voir : 
- Marcel
- Timéo


In [8]:
# On recommence avec Timéo. Il doit apparaître après Françoise.
trajet.insère_patient("Timéo")
print(trajet)

infirmière -> Françoise -> Timéo -> infirmière


In [9]:
trajet.patients_à_visiter()

Patients à voir : 
- Marcel


### Trajet aléatoire

Le langage Python permet très facilement de générer des trajets aléatoires.

On peut récupérer aléatoirement un patient à visiter (donc qui n'a pas encore été vu) avec l'instruction `trajet.patient_aléatoire()`.

In [10]:
# On part d'un trajet vide
trajet = Trajet(données)

# On récupère un patient aléatoire
trajet.patient_aléatoire()

'Françoise'

In [11]:
# On récupère encore un patient aléatoire, exécuter plusieurs fois cette cellule peut donner des résultats différents.
trajet.patient_aléatoire()

'Timéo'

Compléter la fonction `trajet_aléatoire` suivante pour qu’elle retourne un trajet aléatoire :
- il faut récupérer le nombre `p` de patients ;
- initialiser un trajet ;
- faire `p` fois les actions suivantes :
  - trouver un patient aléatoire ;
  - l'ajouter au trajet
- enfin retourner le trajet construit.

In [14]:
def trajet_aléatoire(données: Données) -> Trajet:
    p = données.nombre_patients()
    trajet = Trajet(données)
    for i in range(p):
        patient = trajet.patient_aléatoire()
        trajet.insère_patient(patient)
    return trajet

In [15]:
print(trajet_aléatoire(données))

infirmière -> Françoise -> Timéo -> Marcel -> infirmière


On est en droit de douter de l’efficacité d’un trajet aléatoire ! L’idée est plutôt que cet algorithme fournit une première solution que l’on va chercher à améliorer. Une possibilité est d’échanger deux patients : si la distance totale parcourue est inférieure, on garde cette nouvelle solution, sinon on la rejette. On effectue ces échanges pendant une durée déterminée. On procède en plusieurs étapes.

La distance entre les points $A$ et $B$ de longitudes $\lambda_A$ et $\lambda_B$ respectivement et de latitude $\phi_A$ et $\phi_B$ peut être calculée en première approximation (théorème de Pythagore…) par la formule :
$$d = 1.852 \times 60 \times \sqrt{(\phi_B - \phi_A)^2 + (\lambda_B - \lambda_A)^2 \cos\left(\dfrac{\phi_A + \phi_B}{2}\right)^2}.$$

d’après [ce site](http://villemin.gerard.free.fr/aGeograp/Distance.htm).

Compléter la fonction `distance` qui prend en argument les données géographiques des patients ainsi que les identifiants de deux patients et qui retourne la distance entre ces deux patients. On peut calculer le cosinus d'un angle à l'aide de la fonction `np.cos` et la racine carrée d'un nombre positif avec `np.sqrt` (après avoir importé le module `numpy` avec l'instruction `import numpy as np`).

Attention : la fonction `np.cos` attend un angle en radians, or les latitudes et longitudes sont exprimées en degré. Il faudra donc d'abord convertir les degrés en radians dans le cosinus (mais pas ailleurs !).

In [16]:
import numpy as np

def distance(données: Données, source: Identifiant, cible: Identifiant) -> float:
    (phi_a, lambda_a) = données.coordonnées(source)
    (phi_b, lambda_b) = données.coordonnées(cible)
    return 1.852 * 60 * np.sqrt((phi_b - phi_a)**2 + (lambda_b - lambda_a)**2 * np.cos(np.radians((phi_a + phi_b)/2))**2)

In [17]:
# Calculons la distance entre l'infirmière et Marcel. Regardez sur une carte pour vérifier la crédibilité de ce résultat.

distance(données, "infirmière", "Marcel")

np.float64(1.5441326262890187)

Compléter la fonction `distance_totale`, qui prend en argument les données géographiques des patients ainsi qu’un trajet, et qui retourne la distance parcourue. On peut récupérer le `i`-ème patient vu avec l'instruction `trajet.patient(i)`. On récupère le jeu de données qui a servi à l'initialisation du trajet avec l'instruction `trajet.données` (sans parenthèses).

In [ ]:
def distance_totale(trajet: Trajet) -> float:
    données = trajet.données
    

In [ ]:
# On teste avec un trajet aléatoire.
trajet = trajet_aléatoire(données)
distance_totale(trajet)

L'instruction `trajet.échange(patient1, patient2)` crée un nouveau trajet, obtenu en échangeant les places des patients `patient1` et `patient2`. Le trajet `trajet` quant à lui n'est pas modifié.



In [ ]:
print(trajet)
print(trajet.échange("Marcel", "Françoise"))

Écrire une fonction `améliore(trajet)` qui améliore `trajet` :
- on commence par choisir un patient aléatoirement, appelé `patient1` (attention, l'instruction `trajet.patient_aléatoire` ne marche pas ici car il n'y a plus de patients à visiter, comment faire ?);
- puis on choisit un deuxième patient aléatoirement différent de `patient1`, nommé `patient2` ;
- ensuite on crée le nouveau trajet obtenu en échangeant les places de `patient1` et `patient2` ;
- on compare alors les distances totales des deux trajets ;
- on retourne le meilleur des deux trajets.

In [ ]:
def améliore(trajet: Trajet) -> Trajet:
    

Compléter la fonction `trajet_aléatoire_amélioré`, qui prend en argument les données géographiques des patients et une durée maximale en seconde, qui retourne un trajet, en améliorant un trajet aléatoire par échange aléatoire de deux patients, en limitant la durée d’exécution. On pourra afficher régulièrement la distance du trajet pour vérifier qu'elle décroît avec le temps. Pour cela :
- on récupère l'heure actuelle avec `datetime.now()` que l'on stocke dans la variable `début` (après avoir importé le module `datetime`) ;
- on calcule un trajet aléatoire ;
- on initialise un compteur ;
- tant que la durée qu'il y a entre l'instant actuel et celui fixé dans `début` est inférieur à `temps_max` (on crée une durée en faisant la différence entre deux instants ou avec l'instruction `timedelta(seconds=temps_max)`) ;
    - on remplace le trajet par celui amélioré par un échange ;
    - on incrémente le compteur ;
    - si le compteur est un multiple de 100, on affiche la longueur du trajet ;
- on retourne le trajet en sortie de boucle.

In [ ]:
from datetime import datetime, timedelta

def trajet_aléatoire_amélioré(données: Données, temps_max: int = 30) -> Trajet:
    début = 
    trajet = 
    cpt = 0
    while 

In [ ]:
trajet_aléatoire_amélioré(données, 5)

L'exemple précédent n'est pas intéressant, il y a trop peu de patients. La fonction `données_aléatoires` permet de générer des données aléatoires. Elle a un seul argument : le nombre de patients.

In [ ]:
#Données aléatoires avec 24 patients
données = données_aléatoires(24)
trajet_aléatoire_amélioré(données)

### Approche gloutonne

Plutôt que de laisser faire le hasard, on peut procéder de manière plus pragmatique : quand l’infirmière a fini avec un patient (ou quand elle démarre sa tournée), elle se dirige vers le patient non visité le plus proche d’elle. Cette solution est dite *gloutonne*. Malheureusement on n’est pas assuré de trouver la meilleure solution.

Compléter la fonction `trajet_glouton` :
- il faut récupérer le nombre de patients à voir ;
- initialiser un trajet ;
- construire le trajet en ajoutant à chaque étape le patient le plus proche (donné par l'instruction `trajet.plus_proche_patient()`)
- afficher la distance totale du trajet
- retourner le trajet

In [ ]:
def trajet_glouton(données: Données) -> Trajet:
    

In [ ]:
#Essayons cet algorithme avec les mêmes données que précédemment.
trajet_glouton(données)

Il faut à présent enregistrer les algorithmes qui seront utilisés par l'équipe de présentation des résultats, comme ci-dessous.

In [ ]:
algorithmes = Algorithmes()

algorithmes.ajout_algo(trajet_aléatoire, "trajet aléatoire")
algorithmes.ajout_algo(trajet_aléatoire_amélioré, "trajet aléatoire amélioré")
algorithmes.ajout_algo(trajet_glouton, "méthode gloutonne")

## Tests

Pour vérifier les algorithmes précédents, il faut générer des données géographiques, plus ou moins conséquentes suivant que l’on veut tester les performances ou non de ces algorithmes. Il faut vérifier que les algorithmes 
- retournent des données en accord avec les contraintes posées (un trajet est une liste d’identifiants, qui commence et se termine par INFIRMIERE…)
- ne mettent pas trop de temps à s’exécuter
- fournissent des solutions raisonnables.

On peut générer des données de manière complètement aléatoire, ou chercher des données plus réalistes en consultant des statistiques officielles, ou encore des données destinées à «piéger» les algorithmes.



On peut récupérer les coordonnées géographiques de toutes les adresses postales des communes de Nantes Métropole sur le site [https://data.gouv.fr](https://data.gouv.fr). À partir de ce fichier, il est possible de créer un jeu de données. Pour cela, plusieurs fonctions ont été écrites dans le module Geographie. Une fois importé, il est possible de les utiliser directement. L'idée est que vous vous appropriiez les fonctions de ce module pour créer un jeu de données.

Attention, comme le fichier contient des centaines de milliers d'adresses, il est assez long à charger.

In [ ]:
from Geographie import *

Pour faciliter la saisie d'adresses, un formulaire a été préparé. Pour l'appeler, il suffit d'écrire l'instruction `display(w_adresse)`.

In [ ]:
# Affichage du formulaire
display(w_adresse)

Une fois le formulaire rempli, la fonction `sélection_interactive`, appelée sans argument, extrait du fichier des adresses aléatoirement, correspondant à la commune, quartier, nom de voie et numéro. Attention, si le nombre de patients sélectionnés est inférieur au nombre d'adresses possibles, une erreur sera déclenchée.

In [ ]:
sélection_interactive()

Avec le formulaire, choisir une adresse pour l'infirmière. Dans ce cas, le nombre d'adresses doit valoir 1 ici.

In [ ]:
display(w_adresse)

In [ ]:
adresse_infirmière = sélection_interactive()
adresse_infirmière

On peut vérifier l'adresse sélectionnée sur une carte de Nantes.

In [ ]:
carte_adresses(adresse_infirmière)

Pour construire un jeu de données, l'idée est de réutiliser le formulaire pour sélectionner plusieurs adresses, les stocker dans des variables, et ensuite de les combiner. Ce procédé permet de créer des jeux de données de qualités différentes :
- pour un scénario réaliste, on prendra des patients situés dans un même secteur, proche du domicile de l'infirmière ;
- pour tester les algorithmes de l'équipe de codage, on prendra des patients tantôt très proches, tantôt très éloignés.

On peut combiner les données produites par `sélection_interactive` grâce à la fonction `combine`, qui accepte autant d'arguments que de données à fusionner. On donnera systématiquement en premier argument à cette fonction `adresse_infirmière`.

Exemple :
- Avec le formulaire, je sélectionne Carquefou, allée Molière, 2 adresses, et je crée les données correspondantes dans `données_carquefou`
- puis je sélectionne Nantes, quartier Île de Nantes, 6 adresses, stockées dans `données_nantes` (attention à ne pas revalider la cellule avec `données_carquefou=…` sinon les données de l'étape précédente seront écrasées).
- enfin, je fusionne les deux, et j'affiche sur la carte pour vérifier.

In [ ]:
display(w_adresse)

In [ ]:
données_carquefou = sélection_interactive()

In [ ]:
données_nantes = sélection_interactive()

In [ ]:
adresses = combine(adresse_infirmière, données_carquefou)#, données_nantes)

In [ ]:
carte_adresses(adresses)

Il faut maintenant enregistrer les jeux de données pertinents, pour qu'ils puissent être utilisés par l'équipe de visualisation.

In [ ]:
jeux = Jeux()
#pour chaque jeu de données, on exécute la commande ci-dessous avec à la place de adresses le jeu de données que l'on souhaite exploiter.
jeux.ajout_données(adresses, "8 adresse aléatoire")

Générez à présent autant de jeux de données que vous voulez, puis enregistrez-les en suivant la procédure ci-dessus.

## Visualisation des résultats obtenus

L’équipe de tests a généré des instances et l’équipe de codage les algorithmes. Il faut maintenant présenter ces résultats, de manière graphique, en faisant apparaître les informations pertinentes, et en laissant la possibilité à l’utilisateur de régler différents paramètres utilisés dans le code. C’est là que vous intervenez. Vous utiliserez la bibliothèque `ipywidget`, qui permet d'afficher des «widgets» interactifs. Le choix des widgets vous appartient, à vous de les choisir de sorte à présenter le mieux possible les résultats.


### Sélection du jeu de données

L'équipe de tests a préparé plusieurs jeux de données. La variable `jeux` vous y donne accède. Pour sélectionner un jeu de données, il faut choisir un widget de sélection. Il y en a plusieurs, qui peuvent être personnalisés, voici les possibilités :

In [ ]:
from Selection import *

In [ ]:
# On charge jeux à partir de données bidon
ops = options_bidon #à remplacer quand les équipes de codage et de tests auront terminé par ops = Options(jeux.jeux, algorithmes.algos)
#ops = Options(jeux.jeux, algorithmes.algos)

#Des exemples de widgets permettant une sélection parmi plusieurs choix.
drop = wid.Dropdown()

radio = wid.RadioButtons()

select = wid.Select()

toggle_buttons = wid.ToggleButtons(
    button_style = "success",
)

In [ ]:
#Remplacer drop par les autres widgets pour comparer
selection_jeu_données_w = drop
selection_jeu_données_w.options = ops.jeux_de_données() #ops.jeux_de_données() donne la liste de tous les jeux de données disponibles.
selection_jeu_données_w.description = "Jeu de données"
selection_jeu_données_w.style = {"description_width": "initial"}
display(selection_jeu_données_w)

Le widget `selection_jeu_données_w` s'est affiché ci-dessus à cause de l'instruction `display(selection_jeu_données_w)`.

Une fois le type de widget choisi, on peut changer son apparence. Par exemple pour modifier la taille :

In [ ]:
selection_jeu_données_w.layout = wid.Layout(width="20%", height="50%")

L'équipe de codage aura développé plusieurs algorithmes. Là aussi il faut en sélectionner un. Modifiez le code ci-dessous pour que ça vous convienne.

In [ ]:
selection_algo_w = wid.Dropdown(
    options = ops.algorithmes(),
    description = "Algorithme"
)

display(selection_algo_w)

On peut récupérer la valeur choisie dans un widget avec l'instruction `nom_widget.value`.

In [ ]:
données = selection_jeu_données_w.value
print(données)

Une fois l'algorithme et le jeu de données choisis, on en déduit un trajet. Compléter la fonction `tournée` pour qu'elle retourne ce trajet :
- récupérer l'algorithme et le stocker dans la variable `algo` ;
- récupérer le jeu de données et le stocker dans la variable `données` ;
- appliquer `algo` à `données` (`algo` contient une fonction) et retourner le résultat.

In [ ]:
def tournée():
    algo = 
    données = 
    return 

Le widget `log` défini ci-dessous permet d'afficher les messages produits par l'algorithme dans une fenêtre.

In [ ]:
log = wid.interactive_output(tournée, {})
#display(log)

On peut afficher le trajet calculé sous forme de texte, mais il est plus pertinent de l'afficher sous forme d'un tracé sur une carte. La fonction `tracé` permet de le faire. Chaque segment peut être coloré différemment. Il faut donc choisir ces couleurs : on peut choisir une même couleur pour tous, ou bien un dégradé en fonction de la longueur du segment. La fonction `tracé` accepte comme premier argument le trajet à représenter, puis des paramètres nommés.

In [ ]:
trajet = tournée()
tracé(trajet, couleur_icone_infirmiere="beige", couleur_icones_patients="green", couleur_segments_trajet=["red", "purple"])

Pour comparer les algorithmes, on peut comparer les distances totales parcourues par l'infirmière, et aussi une estimation du temps que ça représente. La distance parcourue a été codée par l'équipe de codage, elle s'appelle `distance_totale(trajet)`, nous pouvons l'utiliser directement. Nous disposons aussi de la fonction `distance` qui calcule la distance entre deux patients : elle s'utilise sous la forme `distance(trajet.données, nom_du_patient1, nom_du_patient2)`.

Écrire une fonction `temps(trajet)` qui estime le temps correspondant à cette distance. Ce temps sera exprimé sous une forme sympathique pour un utilisateur humain, par exemple sous la forme "2 heures et 24 minutes".

In [ ]:
def temps(trajet):
    

Nous allons créer un bouton : quand ce dernier sera cliqué, la carte et la sortie de log seront affichées. Il faut coder la fonction qui sera appelée lorsque le bouton sera cliqué. Compléter la fonction `affiche`:
- `trajet` doit être calculé avec la fonction `tournée` ;
- la fonction `tracé` doit être personnalisée avec vos choix de couleurs ;
- ne modifiez rien d'autre.

In [ ]:
def affiche(bouton):
    with log:
        log.clear_output()
        trajet = 
    with info:
        d = distance_totale(trajet)
        t = temps(trajet)
        print(f"Distance totale parcourue : {d}")
        print(f"Estimation du temps de route : {t}" )
    with carte:
        carte.clear_output()
        m = tracé(trajet,couleur_icone_infirmiere=, couleur_icones_patients=, couleur_segments_trajet=)
        display(m)

bouton = wid.Button(
    description = "Afficher la carte",
)

bouton.on_click(affiche)

carte = wid.Output()
info = wid.Output()

Pour présenter les résultats, nous définissons ci-dessous un widget qui permet de placer les widgets déjà définis. Vous pouvez là aussi modifier l'ordre des widgets, ou modifier ces widgets comme vu précédemment.


In [ ]:
interface_démonstration = wid.AppLayout(
    left_sidebar = wid.VBox([selection_algo_w, selection_jeu_données_w, bouton]),
    center = carte,
    right_sidebar = log,
    header = wid.Label("Présentation"),
    footer = info,
    pane_widths = [1,3,1],
    pane_heights = [1,5,0]
)

# Réglage éventuel des widgets



In [ ]:
display(interface_démonstration)